In [1]:
import torch
import numpy as np
import torch.nn as nn

In [2]:
import importnb
with __import__('importnb').Notebook(): 
    from tools import ScaledDotProductAttention
    from tools import MultiHeadAttention

## SelfAttentionの実行

### 適当なデータの作成
---
新しくデータkを作る。kは***(B,T,d_model)***のshapeを持つテンソルである。


### SelfAttentionにする
---
q,k,vを同じテンソルにすることでSelfAttentionにする。


### forwardで計算を行う。
---
呼び出したspa.forward()によって計算を行う。
この時spaの初期化で与えるd_kには本来d_modelをhead数で割った値が入る(デフォルトだと512/8で64)

In [11]:
###SelfAttentionの実行
num_head = 4
batch_size = 2
seq_len = 10
d_model = 64
k = torch.randn(batch_size,seq_len,d_model)

In [12]:
###SelfAttentionにするためにqとvとkを一緒にする
q = torch.randn(batch_size,seq_len,d_model)
v = torch.randn(batch_size,seq_len,d_model)
spa = ScaledDotProductAttention(16)
attention_weight = spa.forward(q,k,v)

In [13]:
attention_weight.shape

torch.Size([2, 10, 64])

## MultiHeadAttentionの実行
ランダムなテンソル、スタンダードなマスクを使う。

In [14]:
def create_incremental_mask(seq_len):
    """
    seq_len x seq_len のサイズのマスクを生成する。
    0列目は全てFalse、以降の列では上から順にTrueの数を増やしていく。
    """
    # seq_len x seq_len の行列を生成し、初期値は全てFalseに設定
    mask = torch.full((seq_len, seq_len), False)

    # 各列に対して、上から順にTrueをセットする
    for i in range(seq_len):
        mask[:i, i] = True

    return mask
mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
mask.shape

torch.Size([2, 10, 10])

In [63]:
mha = MultiHeadAttention(num_head,d_model)
output = mha(q,k,v,mask=mask)

In [64]:
output.shape

torch.Size([2, 10, 64])

In [65]:
output

tensor([[[ 0.0957, -0.1196, -0.0282,  ..., -0.0597, -0.2022,  0.0933],
         [ 0.0520, -0.1581, -0.1929,  ..., -0.1734, -0.1570, -0.1558],
         [ 0.0274, -0.1966, -0.1885,  ..., -0.1473, -0.1109, -0.0960],
         ...,
         [ 0.0266, -0.1267, -0.1294,  ..., -0.1157, -0.1193, -0.0491],
         [-0.0135, -0.0415, -0.0956,  ..., -0.1137, -0.1243, -0.0560],
         [-0.0122, -0.0608, -0.0876,  ..., -0.1133, -0.1376, -0.0450]],

        [[-0.3005, -0.1087, -0.0952,  ...,  0.0574, -0.0104,  0.0695],
         [-0.0461, -0.1701,  0.0091,  ...,  0.0523, -0.1382, -0.0243],
         [-0.0439, -0.0755, -0.0062,  ...,  0.0881, -0.0864, -0.0440],
         ...,
         [ 0.0058, -0.0382,  0.0456,  ..., -0.0073, -0.0369,  0.0097],
         [ 0.0018,  0.0047,  0.0597,  ..., -0.0319, -0.0606,  0.0176],
         [ 0.0194,  0.0180,  0.0563,  ..., -0.0228, -0.0629,  0.0430]]],
       grad_fn=<ViewBackward0>)